In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from typing import List

import time

from statistics import mean

from xgboost import XGBClassifier

from catboost import CatBoostClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import silhouette_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.ensemble  import StackingClassifier

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier 

from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA

import optuna as op


C:\Users\Mate\AppData\Roaming\Python\Python39\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
df = pd.read_pickle('../data/processed/train.pkl')
df

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,-0.6209,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,-3.6797,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,0.1202,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995


In [3]:
cols_float = df.select_dtypes(include=["float64"]).columns.tolist() 
df[cols_float] = df[cols_float].apply(pd.to_numeric, downcast='float')
df.dtypes

target       int64
var_0      float32
var_1      float32
var_2      float32
var_3      float32
            ...   
var_195    float32
var_196    float32
var_197    float32
var_198    float32
var_199    float32
Length: 201, dtype: object

In [4]:
X=df.drop("target", axis=1)
y=df["target"]

In [5]:
skf = StratifiedKFold(n_splits=7,shuffle=False)
skf.get_n_splits(X, y)

7

In [6]:
#Genero lista de resultados
results=[]
lista_name=[]
tiempo_ejecucion=[]

In [10]:
gnb = GaussianNB()
gnb2 = GaussianNB()
gnb3 = GaussianNB()
gnb4 = GaussianNB()
gnb5 = GaussianNB()

gnb_final = GaussianNB()

In [11]:
ensamble = StackingClassifier(estimators=[("gnb",gnb),("gnb2",gnb2),("gnb3",gnb3),("gnb4",gnb4),("gnb5",gnb5)],final_estimator=gnb_final)

In [12]:
#==================================Stackin 5 Gaussian Naïve Bayes================================================
contador=0
inicio=time.time()
recall_prom=[]

for train_index, test_index in skf.split(X, y):
    
    X_train=X.iloc[train_index]
    X_test=X.iloc[test_index]
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    
    ensamble.fit(X_train, y_train)
    
    y_pred=np.asarray(ensamble.predict(X_test))
    
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')    
    recall = recall_score(y_test, y_pred,average='binary')
    recall_prom.append(recall)
    
    if contador==2:
        print(cm,precision,recall)
    contador+=1
    
results.append(mean(recall_prom))

print("El valor promedio del recall es", mean(recall_prom))

lista_name.append("Fulldata_ensamble_Gaussian_Naïve_Bayes")

fin = time.time()
tiempo_ejecucion.append(fin-inicio)
print(f"el tiempo fue de {fin-inicio:.2f}")

[[23859  1841]
 [ 1181  1691]] 0.47876557191392977 0.5887883008356546
El valor promedio del recall es 0.6040907205900963
el tiempo fue de 122.84


In [7]:
gnb = GaussianNB()
gnb2 = GaussianNB()
gnb3 = GaussianNB()
gnb4 = GaussianNB()
gnb5 = GaussianNB()

lg=LogisticRegression(max_iter= 23980, penalty="none", solver= 'lbfgs')

gnb_final = GaussianNB()

In [8]:
ensamble = StackingClassifier(estimators=[("gnb",gnb),("gnb2",gnb2),("gnb3",gnb3),("gnb4",gnb4),("gnb5",gnb5),("lg",lg)],final_estimator=gnb_final)

In [9]:
#==============================Stackin 5 Gaussian Naïve Bayes + Logistic Regression===================================
contador=0
inicio=time.time()
recall_prom=[]

for train_index, test_index in skf.split(X, y):
    
    X_train=X.iloc[train_index]
    X_test=X.iloc[test_index]
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    
    ensamble.fit(X_train, y_train)
    
    y_pred=np.asarray(ensamble.predict(X_test))
    
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')    
    recall = recall_score(y_test, y_pred,average='binary')
    recall_prom.append(recall)
    
    if contador==2:
        print(cm,precision,recall)
    contador+=1
    
results.append(mean(recall_prom))

print("El valor promedio del recall es", mean(recall_prom))

lista_name.append("Fulldata_ensamble_Gaussian_Naïve_Bayes")

fin = time.time()
tiempo_ejecucion.append(fin-inicio)
print(f"el tiempo fue de {fin-inicio:.2f}")

[[23816  1884]
 [ 1166  1706]] 0.47520891364902507 0.5940111420612814
El valor promedio del recall es 0.6081209130147752
el tiempo fue de 3704.59
